In [1]:
import tensorflow as tf
tf.config.list_physical_devices()

2022-12-12 00:43:28.847744: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-12 00:43:34.980319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 00:43:34.986640: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 00:43:34.987237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
!git clone https://github.com/tirthankar95/CSCI_7000_FinalProject.git

Cloning into 'CSCI_7000_FinalProject'...
remote: Enumerating objects: 687, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 687 (delta 88), reused 173 (delta 57), pack-reused 477
Receiving objects: 100% (687/687), 26.90 MiB | 46.07 MiB/s, done.
Resolving deltas: 100% (293/293), done.


In [5]:
%cd CSCI_7000_FinalProject/gym-minigrid/

/home/jovyan/examples/examples/tensorflow/CSCI_7000_FinalProject/gym-minigrid


In [6]:
import keras

In [5]:
import matplotlib
import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [7]:
!pip install tqdm

  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)


In [8]:
!pip install -e .

Obtaining file:///home/jovyan/examples/examples/tensorflow/CSCI_7000_FinalProject/gym-minigrid
  Preparing metadata (setup.py) ... done
  Using cached gym-0.25.2-py3-none-any.whl
  Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
  Running setup.py develop for gym-minigrid


In [9]:
from gym_minigrid.envs.doorkey import *
from gym_minigrid.envs.crossing import *
from gym_minigrid.envs.mixed import *
#env = CrossingEnv(size=6)
env = MixedEnv(size=8)
env.step_m(1)

/home/jovyan/examples/examples/tensorflow/CSCI_7000_FinalProject/gym-minigrid/gym_minigrid/roomgrid.py:300: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if front_cell is None or front_cell.type is 'wall':
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/gym/utils/seeding.py:63: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(
/home/jovyan/examples/examples/tensorflow/CSCI_7000_FinalProject/gym-minigrid/gym_minigrid/minigrid.py:663: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.ze

({'image': array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [4, 4, 2],
          [2, 5, 0]],
  
         [[2, 5, 0],
          [1, 0, 0],
          [1, 0, 0],
          [5, 4, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0]],
  
         [[2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
 

In [10]:
import gym
import numpy as np
import random
import gym_minigrid
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential,Model
from keras.layers import LSTM,Bidirectional,Dense,Input,Embedding,TimeDistributed
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [11]:
num_actions=5

In [12]:

def plotProgress(reward_plot):
    plt.plot(reward_plot)
    plt.xlabel('Episodes')
    plt.ylabel('Avg. reward')
    plt.title('Avg Reward Per Step V/S Episodes.')
    plt.show()


In [13]:
frame_offset = 3

In [14]:
def create():
    global num_actions
    input=Input(shape=((frame_offset+1),147)) # (3,3,7,7) ~ the mini-grid by default returns (3,7,7) image.
    model=LSTM(units=256,return_sequences=False)(input)
    x1 = Dense(units=256, activation='relu')(model)
    x1 = Dense(units=128, activation='relu')(x1)
    x1 = Dense(units=64, activation='relu')(x1)
    x1 = Dense(units=32, activation='relu')(x1)
    x1 = Dense(units=16, activation='relu')(x1)
    output=Dense(units=num_actions,activation='linear')(x1)
    model=Model(input,output)
    return model


In [15]:
create()

2022-12-12 00:45:28.948671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 00:45:28.949139: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 00:45:28.949839: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 00:45:28.950420: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so ret

In [24]:
from tqdm import tqdm
global num_actions
# Configuration paramaters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 0.8  # Epsilon greedy parameter
epsilon_min = 0.2  # Minimum epsilon greedy parameter
epsilon_max = 0.8  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 4096  # Size of batch taken from replay buffer
max_steps_per_episode = 5000 #beast 1000
from gym_minigrid.envs.doorkey import *
from gym_minigrid.envs.crossing import *
from gym_minigrid.envs.mixed import *
# env = DoorKeyEnv(size=8)
#env = CrossingEnv(size=6)
env = MixedEnv(siz=8)
#env = gym.make('MiniGrid-DoorKey-6x6-v0')
env.seed(seed)

mixed_model=create()
mixed_model_target=create()
loss_function = keras.losses.MeanSquaredError()
optimizer=keras.optimizers.RMSprop()

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
episode_reward_history=[]
done_history = []
reward_plot = []
running_reward = 0
episode_count = 0
frame_count = 0
# Number of frames to take random action and observe output
epsilon_random_frames = 100000 #beast 10000
# Number of frames for exploration
epsilon_greedy_frames = 500000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 200000
# Train the model after 4 actions
update_after_actions = 100
# How often to update the target network
update_target_network = 10000
# Using huber loss for stability
# We are taking 3 frames in our LSTM
#frame_offset=2



In [25]:
termination_steps=[]

In [26]:
success_ratio = []
epsilon_history = [epsilon]

In [27]:
import numpy as np
seed = 42
np.random.seed(seed=42)
def best_action(action_probs):
  action = np.argmax(action_probs)
  action_probs = np.abs(action_probs - action_probs[action])
  possible_actions = []
  for i in range(5):
    if (action_probs[i] < 0.2):
      possible_actions.append(i)
  # print(action_probs, possible_actions)
  length = len(possible_actions)
  action_probs = [1 / length for i in possible_actions]
  return np.random.choice(possible_actions, p=action_probs)


In [28]:
mixed_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 4, 147)]          0         
                                                                 
 lstm_3 (LSTM)               (None, 256)               413696    
                                                                 
 dense_18 (Dense)            (None, 256)               65792     
                                                                 
 dense_19 (Dense)            (None, 128)               32896     
                                                                 
 dense_20 (Dense)            (None, 64)                8256      
                                                                 
 dense_21 (Dense)            (None, 32)                2080      
                                                                 
 dense_22 (Dense)            (None, 16)                528 

In [59]:
mixed_agent_targets = []
mixed_agent_targets.append(keras.models.load_model("DoorKey_Experts1"))
mixed_agent_targets.append(keras.models.load_model("Lava_Experts2"))

In [60]:
max_steps = 5000
episodes = 100
experts_rewards = []
for e in range(len(mixed_agent_targets)):
    avg_reward = 0
    for i in range(episodes):
        state = np.array(env.reset_m())
        episode_reward = 0
        temp_state=dict(state.item(0)) # state is a 0-d numpy array.
        state_history = [temp_state['image']]
        for i in range(frame_offset):
            action = np.random.choice(5)
            state, reward, done, _ = env.step_m(action)
            state_numpy = np.array(state)
            temp_state = dict(state_numpy.item(0))
            state_history.append(temp_state['image'])
        #print(np.array(state_history[-8:]).shape)
        for j in range(max_steps):
            state_numpy = np.array(state_history[-(frame_offset+1):])
            #print(state_numpy.shape)
            state_numpy = state_numpy.reshape((frame_offset+1),147)
            state_numpy = np.array([state_numpy])
            action_probs = mixed_agent_targets[e](state_numpy, training=False)
            action = np.argmax(action_probs[0])
            state, reward, done, _ = env.step_m(action)
            state_numpy = np.array(state)
            temp_state = dict(state_numpy.item(0))
            del state_history[:1]
            state_history.append(temp_state['image'])
            # state_history.append(state)
            #print(done,action)
            episode_reward += reward
            if done:
                break
        episode_avg_reward = episode_reward/j
        avg_reward += episode_avg_reward
        print(e,episode_avg_reward,j,episode_reward)
    experts_rewards.append(avg_reward/episodes)


0 -7.411764705882353 17 -126
0 -11.5 10 -115
0 -8.785714285714286 14 -123
0 -8.785714285714286 14 -123
0 -8.266666666666667 15 -124
0 -8.785714285714286 14 -123
0 -10.909090909090908 11 -120
0 -11.9 10 -119
0 -10.083333333333334 12 -121
0 -7.411764705882353 17 -126
1 -1.0002000400080016 4999 -5000
1 -1.0002000400080016 4999 -5000
1 -1.0002000400080016 4999 -5000
1 -1.0002000400080016 4999 -5000
1 -1.0002000400080016 4999 -5000
1 -1.0002000400080016 4999 -5000
1 -1.0002000400080016 4999 -5000
1 -1.0002000400080016 4999 -5000
1 -1.0002000400080016 4999 -5000
1 -1.0002000400080016 4999 -5000


In [61]:
experts_rewards

[-9.383976317799846, -1.0002000400080016]

In [ ]:

def softmax(arr):
  for i in range(len(arr)):
    print(np.exp(arr[i])/np.sum(np.exp(arr)))
weights = softmax(experts_rewards)

In [ ]:

def updateNovice(novice,exp,rewardArr):
  m=novice.get_weights()
  wt=[]
  for xx in range(len(m)):
    temp=0
    for _ in range(N_EXPERTS):
      temp+=np.array(exp[_].get_weights()[xx])*rewardArr[_]
    wt.append(temp)
  novice.set_weights(wt)

novice=create()
updateNovice(novice,mixed_agent_targets,rewardArr)

In [ ]:
novice_target = create()

In [ ]:
noOfEpisodes=100 #beast 100000
#while noOfEpisodes:  # Run until solved
for _ in tqdm(range(noOfEpisodes)):
    #noOfEpisodes-=1
    state = np.array(env.reset_m())
    episode_reward = 0
    success = 0

    for timestep in range(1, max_steps_per_episode):
        # env.render(); Adding this line would show the attempts
        # of the agent in a pop up window.
        frame_count += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            #print(len(state_history))
            state_numpy = np.array(state_history[-(frame_offset+1):]).reshape((frame_offset+1),147)
            state_numpy = np.array([state_numpy])
            action_probs = novice(state_numpy, training=False)
            # Take best action
            #action = np.argmax(action_probs[0])
            action = best_action(action_probs[0])
            #print(action_probs[0])
            

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)
        #epsilon_history.append(epsilon)
        

        # Apply the sampled action in our environment
        state_next, reward, done, _ = env.step_m(action)
        state_next = np.array(state_next)

        episode_reward += reward
        if(done==True):
            success = 1

        # Save actions and states in replay buffer
        action_history.append(action)
        temp_state=dict(state.item(0)) # state is a 0-d numpy array.
        state_history.append(temp_state['image'])
        temp_state=dict(state_next.item(0)) # state is a 0-d numpy array.
        state_next_history.append(temp_state['image'])
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)-frame_offset), size=batch_size)+frame_offset
            i=indices[0]
            # Using list comprehension to sample from replay buffer
            state_sample = np.array([ np.array(state_history[i-frame_offset:i+1]).reshape((frame_offset+1),147) for i in indices])
            state_next_sample = np.array([ np.array(state_next_history[i-frame_offset:i+1]).reshape((frame_offset+1),147) for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = novice_target.predict(state_next_sample,verbose=False)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * np.max(\
                future_rewards, axis=1)
            updated_q_values = updated_q_values.astype('float32')
            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)
            with tf.GradientTape() as tape:    
                q_values = novice(state_sample)
                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                loss = loss_function(updated_q_values,q_action)
            # Backpropagation
            grads = tape.gradient(loss, novice.trainable_variables)
            optimizer.apply_gradients(zip(grads, novice.trainable_variables))
            

        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            novice_target.set_weights(novice.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {} with epsilon {}"
            print(template.format(running_reward, episode_count, frame_count, epsilon))
        if frame_count > update_target_network*10:
            update_target_network = update_target_network*10

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]
        if done:
            break

    success_ratio.append(success)
    # Update running reward to check condition for solving
    epsilon_history.append(epsilon)
    episode_reward_history.append(episode_reward)
    running_reward = np.mean(episode_reward_history)
    reward_plot.append(episode_reward/timestep)
    episode_count += 1
    termination_steps.append(timestep)


In [ ]:
novice.save("MixedTransferLearning")